# Problem 3B (LSTM CELL)

In [ ]:
"""
Name: Saif Khan
Objective: To create a Basic LSTM Cell for Part 3B

"""


import tensorflow as tf 
from tensorflow.python.ops import rnn, rnn_cell
import numpy as np 
import matplotlib.pyplot as plt

if(tf.__version__.split('.')[0]=='2'):
    import tensorflow.compat.v1 as tf
    tf.disable_v2_behavior()    

# Load MNIST dataset
import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)


learningRate = 0.001
trainingIters = 30000
batchSize = 100 #Set to 128,256 or 512 to change iterations
displayStep = 10

nInput = 28 #we want the input to take the 28 pixels
nSteps = 28 #every 28
nHidden = 512 #number of neurons for the RNN
nClasses = 10 #this is MNIST so you know

x = tf.placeholder('float', [None, nSteps, nInput])
y = tf.placeholder('float', [None, nClasses])

weights = {
	'out': tf.Variable(tf.random_normal([nHidden, nClasses]))
}

biases = {
	'out': tf.Variable(tf.random_normal([nClasses]))
}

def RNN(x, weights, biases):
	x = tf.transpose(x, [1,0,2])
	x = tf.reshape(x, [-1, nInput])
	x = tf.split(x, nSteps, 0) #configuring so you can get it as needed for the 28 pixels


	lstmCell = rnn_cell.BasicLSTMCell(nHidden)
	

	outputs, states = tf.compat.v1.nn.static_rnn(lstmCell, x, dtype = tf.float32)

	return tf.matmul(outputs[-1], weights['out'])+ biases['out']

pred = RNN(x, weights, biases)

#optimization
#create the cost, optimization, evaluation, and accuracy
#for the cost softmax_cross_entropy_with_logits seems really good
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learningRate).minimize(cost)

correctPred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))

init = tf.initialize_all_variables()
#Extra stuff for plotting
accs_list = []
losses_list = []
testacc_list = []
with tf.Session() as sess:
	sess.run(init)
	step = 1

	testData = mnist.test.images.reshape((-1, nSteps, nInput))
	testLabel = mnist.test.labels
	while step* batchSize < trainingIters:
		batchX, batchY = mnist.train.next_batch(batchSize) #mnist has a way to get the next batch
		batchX = batchX.reshape((batchSize, nSteps, nInput))

		sess.run(optimizer, feed_dict={x: batchX, y: batchY})

		if step % displayStep == 0:
			acc = sess.run(accuracy, feed_dict={x: batchX, y: batchY})
			accs_list.append(acc)
			loss = sess.run(cost, feed_dict={x: batchX, y: batchY})
			losses_list.append(loss)
			testacc = sess.run(accuracy, feed_dict={x: testData, y: testLabel})
            
			print("Iter " + str(step*batchSize) + ", Minibatch Loss= " + "{:.6f}".format(loss) + ", Training Accuracy= " + "{:.5f}".format(acc) + ", Test Accuracy= " + "{:.5f}".format(testacc))
		step +=1
	print('Optimization finished')

	print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: testData, y: testLabel}))

sess.close()
    
fig, ax = plt.subplots()
ax.plot(range(len(accs_list)), accs_list, 'k', label='GRU Accuracy')
ax.legend(loc='upper right', shadow=True)
plt.savefig('./LSTM Accuracy.png')
plt.show()

fig, bx = plt.subplots()
bx.plot(range(len(losses_list)), losses_list, 'k', label='GRU Loss')
bx.legend(loc='upper right', shadow=True)
plt.savefig('./LSTM Accuracy.png')
plt.show()

